**Install the libraries**

In [1]:
!pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.6 MB/s eta 0:00:00


In [2]:
from collections import defaultdict
import gdown
import gzip
import json
import random
import pickle

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import ticker
sns.set(style='ticks', font_scale=1.2)

In [3]:
#import the libraries
import numpy as np
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#load and read the training TEXT From file
trainingSet = pd.read_csv('drive/MyDrive/spanishOriginal.txt', on_bad_lines='skip', sep=';', header=None)

In [6]:
#show the head of the new CSV file
trainingSet.head()

,0,1,2,3,4
0,id=828025263321657348,||,Ismael es egocentrico porque se vuelve loca si...,||,0
1,id=828025128797741057,||,..ya tardaba en salir quien pronunciase nombre...,||,0
2,id=828025087815274496,||,"(Esto no es un discurso político y razonado, o...",||,0
3,id=828025006626058241,||,"Muy despreciados,siiii,pero todos vestidos de ...",||,1
4,id=828024709761658880,||,marica explicame porque a veces no te entiendo...,||,1


In [7]:
trainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5851 entries, 0 to 5850
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5851 non-null   object
 1   1       5851 non-null   object
 2   2       5851 non-null   object
 3   3       5851 non-null   object
 4   4       5851 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 228.7+ KB


In [8]:
#drop the columns not needed
trainingSet.drop(columns=0, inplace=True)
trainingSet.drop(columns=1, inplace=True)
trainingSet.drop(columns=3,inplace=True)

In [9]:
trainingSet.head()

,2,4
0,Ismael es egocentrico porque se vuelve loca si...,0
1,..ya tardaba en salir quien pronunciase nombre...,0
2,"(Esto no es un discurso político y razonado, o...",0
3,"Muy despreciados,siiii,pero todos vestidos de ...",1
4,marica explicame porque a veces no te entiendo...,1


In [10]:
#rename the column 2 to 'Tweet' and column 4 to 'Label
trainingSet=trainingSet.rename(columns={2: 'Tweet'})
trainingSet=trainingSet.rename(columns={4: 'label'})

In [11]:
trainingSet.head()

,Tweet,label
0,Ismael es egocentrico porque se vuelve loca si...,0
1,..ya tardaba en salir quien pronunciase nombre...,0
2,"(Esto no es un discurso político y razonado, o...",0
3,"Muy despreciados,siiii,pero todos vestidos de ...",1
4,marica explicame porque a veces no te entiendo...,1


**The Perreira Corpus is already binarized, no need to binarize then!**
It contains 5851 tweets and labels with 0 or 1 (0 means no-hate speech, 1 signifies hate speech)





In [12]:
trainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5851 entries, 0 to 5850
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   5851 non-null   object
 1   label   5851 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 91.5+ KB


In [13]:
count = (trainingSet['label'] == 1).sum()

# The condition (df['Label'] == 1) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 1 in 'Label' column:", count)

Number of rows with value 1 in 'Label' column: 1536


In [14]:
countZero = (trainingSet['label'] == 0).sum()

# The condition (df['Label'] == 0) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 0 in 'Label' column:", countZero)

Number of rows with value 0 in 'Label' column: 4315


**Processing and Cleaning the Tweets**

In [15]:
#import libraries for processing
from nltk.stem import WordNetLemmatizer
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re

1. Remove all links from the Tweets

In [16]:
#function remove_urls cleans the Urls from the Tweets and put in a seperate column
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

trainingSet["tweet_w/o_url"] = trainingSet["Tweet"].apply(lambda text: remove_urls(text))

In [17]:
trainingSet.head()

,Tweet,label,tweet_w/o_url
0,Ismael es egocentrico porque se vuelve loca si...,0,Ismael es egocentrico porque se vuelve loca si...
1,..ya tardaba en salir quien pronunciase nombre...,0,..ya tardaba en salir quien pronunciase nombre...
2,"(Esto no es un discurso político y razonado, o...",0,"(Esto no es un discurso político y razonado, o..."
3,"Muy despreciados,siiii,pero todos vestidos de ...",1,"Muy despreciados,siiii,pero todos vestidos de ..."
4,marica explicame porque a veces no te entiendo...,1,marica explicame porque a veces no te entiendo...


In [18]:
trainingSet.head()

,Tweet,label,tweet_w/o_url
0,Ismael es egocentrico porque se vuelve loca si...,0,Ismael es egocentrico porque se vuelve loca si...
1,..ya tardaba en salir quien pronunciase nombre...,0,..ya tardaba en salir quien pronunciase nombre...
2,"(Esto no es un discurso político y razonado, o...",0,"(Esto no es un discurso político y razonado, o..."
3,"Muy despreciados,siiii,pero todos vestidos de ...",1,"Muy despreciados,siiii,pero todos vestidos de ..."
4,marica explicame porque a veces no te entiendo...,1,marica explicame porque a veces no te entiendo...


In [19]:
X = trainingSet["tweet_w/o_url"].values.tolist()

In [20]:
y = trainingSet['label'].values.tolist()

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 0)

In [22]:
train_texts = X_train
train_labels = y_train

test_texts = X_test
test_labels = y_test

In [23]:
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(4680, 4680, 1171, 1171)

In [24]:
train_labels[0], train_texts[0]

(0,
 'Este GH es el más loco del puto mundo jajajajajaja estoy loca ya y me río sola con todo!! #vipdirecto')

In [25]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

In [26]:
#import svm model and metrics
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [27]:

#SVM BASE LINE

model = SVC(kernel='linear', gamma='auto').fit(X_train, train_labels)
predictions = model.predict(X_test)

In [28]:
print(classification_report(test_labels, predictions))

              precision    recall  f1-score   support

           0       0.83      0.94      0.88       870
           1       0.70      0.43      0.53       301

    accuracy                           0.81      1171
   macro avg       0.76      0.68      0.71      1171
weighted avg       0.79      0.81      0.79      1171



In [29]:
#calculate F1 score
from sklearn.metrics import f1_score
f1_score(test_labels, predictions)

0.5338809034907598

In [30]:
from sklearn.metrics import matthews_corrcoef
#calculate Matthews correlation coefficient
matthews_corrcoef(test_labels, predictions)

0.43939610514703514

Repeat the same with SpanishBERT and SpanishData

In [31]:
# This is the name of the BERT model that we want to use.
model_name = 'dccuchile/bert-base-spanish-wwm-cased'

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'

# This is the maximum number of tokens in any document sent to BERT.
max_length = 512

# This is the name of the directory where we'll save our model. You can name it whatever you want.
cached_model_directory_name = 'MasterSpanishBert-saved'

In [32]:
from transformers import BertForSequenceClassification, BertTokenizer
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
#tokenizer = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.

In [33]:
train_labels

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,


In [34]:
unique_labels = set(label for label in train_labels)
label2id = {label: id for id, label in enumerate(unique_labels)}
id2label = {id: label for label, id in label2id.items()}

In [35]:
label2id.keys()

dict_keys([0, 1])

In [36]:
id2label.keys()

dict_keys([0, 1])

In [37]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
test_encodings  = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)

train_labels_encoded = [label2id[y] for y in train_labels]
test_labels_encoded  = [label2id[y] for y in test_labels]

In [38]:
set(train_labels_encoded)

{0, 1}

In [39]:
set(test_labels_encoded)

{0, 1}

In [40]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [41]:
train_dataset = MyDataset(train_encodings, train_labels_encoded)
test_dataset = MyDataset(test_encodings, test_labels_encoded)

In [42]:
# The model_name needs to match the name used for the tokenizer above.
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

In [43]:
#!pip install --upgrade accelerate

In [44]:
#!pip uninstall -y transformers accelerate
#!pip install transformers accelerate

In [45]:
training_args = TrainingArguments(
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)

In [46]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [47]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset (usually a validation set; here we just send our test set)
    compute_metrics=compute_metrics      # our custom evaluation function
)

In [48]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.542800,0.511768,0.762596
200,0.493200,0.457444,0.806149
300,0.429200,0.414663,0.818958
400,0.285600,0.435210,0.821520
500,0.312600,0.386427,0.824082
600,0.262000,0.463937,0.828352
700,0.104400,0.636400,0.823228
800,0.093400,0.746733,0.837746


TrainOutput(global_step=879, training_loss=0.29724656952390355, metrics={'train_runtime': 1272.6949, 'train_samples_per_second': 11.032, 'train_steps_per_second': 0.691, 'total_flos': 3694079217254400.0, 'train_loss': 0.29724656952390355, 'epoch': 3.0})

In [49]:
trainer.save_model(cached_model_directory_name)

In [50]:
trainer.evaluate()

{'eval_loss': 0.7226523160934448,
 'eval_accuracy': 0.8334756618274979,
 'eval_runtime': 7.0293,
 'eval_samples_per_second': 166.589,
 'eval_steps_per_second': 8.393,
 'epoch': 3.0}

In [51]:
predicted_results = trainer.predict(test_dataset)

In [52]:
predicted_results.predictions.shape

(1171, 2)

In [53]:
predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

In [54]:
len(predicted_labels)

1171

Print the Results for the SpanishData and SpanishBert aka BETO

In [55]:
print(classification_report(test_labels,
                            predicted_labels))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       870
           1       0.69      0.65      0.67       301

    accuracy                           0.83      1171
   macro avg       0.78      0.77      0.78      1171
weighted avg       0.83      0.83      0.83      1171



In [56]:
#calculate F1 score
from sklearn.metrics import f1_score
f1_score(test_labels, predicted_labels)

0.667802385008518

In [57]:
from sklearn.metrics import matthews_corrcoef
#calculate Matthews correlation coefficient
matthews_corrcoef(test_labels, predicted_labels)

0.5571119477676865